In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("./modele_team1")

from preprocessing import *
from features import *
from embeddings import *
from models import *
from ensemble import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

E0000 00:00:1744122769.867515    4718 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744122769.872259    4718 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744122769.886157    4718 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744122769.886169    4718 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744122769.886170    4718 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744122769.886172    4718 computation_placer.cc:177] computation placer already registered. Please check linka

In [3]:
df = pd.read_csv("../datasets/training-v1/offenseval-training-v1.tsv", sep='\t')
df.columns = ["id", "text", "label_A", "label_B", "label_C"]

df_b = df[df["label_A"] == "OFF"]
texts = df_b["text"].values
labels = df_b["label_B"].map({"TIN": 1, "UNT": 0}).values

In [4]:
# 3. Préparation des données
norm_texts = [normalize_text(t) for t in texts]


In [5]:
X_feat = extract_features(norm_texts)


In [6]:
# === 4. Embeddings fastText (limités à 50k mots pour la RAM) ===
print("[INFO] Chargement des embeddings fastText...")
embeddings = load_fasttext("../embeddings/crawl-300d-1M.vec", max_words=50000)
X_embed = np.vstack([sentence_to_embedding(t, embeddings) for t in norm_texts])


[INFO] Chargement des embeddings fastText...


In [7]:
# === 5. Fusion features + embeddings ===
X = np.hstack([X_embed, X_feat])


In [8]:
#  === 6. Split train/test ===
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=42)


In [9]:
# === 7. Modèle Random Forest ===
rf = RFModel()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("\n=== Rapport Random Forest ===")
print(classification_report(y_test, y_pred_rf, digits=3))



=== Rapport Random Forest ===
              precision    recall  f1-score   support

           0      0.333     0.056     0.096       160
           1      0.883     0.984     0.931      1160

    accuracy                          0.872      1320
   macro avg      0.608     0.520     0.514      1320
weighted avg      0.817     0.872     0.830      1320



In [10]:
# === 8. Modèle SVM ===
svm = SVMModel()
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
print("\n=== Rapport SVM ===")
print(classification_report(y_test, y_pred_svm, digits=3))



=== Rapport SVM ===
              precision    recall  f1-score   support

           0      0.000     0.000     0.000       160
           1      0.879     1.000     0.935      1160

    accuracy                          0.879      1320
   macro avg      0.439     0.500     0.468      1320
weighted avg      0.772     0.879     0.822      1320



/home/DeepL/Project/deepl/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/DeepL/Project/deepl/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/DeepL/Project/deepl/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [11]:
# === 9. Ensemble Voting (RF + SVM) ===
ensemble = SoftVotingEnsemble([
    ('rf', rf.model),
    ('svm', svm.model)
])
ensemble.fit(X_train, y_train)
y_pred_ens = ensemble.predict(X_test)
print("\n=== Rapport Ensemble (RF + SVM) ===")
print(classification_report(y_test, y_pred_ens, digits=3))



=== Rapport Ensemble (RF + SVM) ===
              precision    recall  f1-score   support

           0      0.333     0.006     0.012       160
           1      0.879     0.998     0.935      1160

    accuracy                          0.878      1320
   macro avg      0.606     0.502     0.474      1320
weighted avg      0.813     0.878     0.823      1320



IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
